#### Functions (IGNORE for Readability)

In [19]:
# import packages that will be used for analysis
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import plotly.graph_objects as go
from dash import dcc,html,Output,Input
import dash

##### Get Stock Data

In [4]:
missing_data_tickers = [] # use this as a list of tickers with missing data

def get_data_from_start_to_end(ticker, start_date, end_date):
    global missing_data_tickers  # Use the global list to accumulate missing tickers
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if stock_data.empty:
            missing_data_tickers.append(ticker)
            raise ValueError(f"Stock data for ticker {ticker} during the period from {start_date} to {end_date} was not found.")
        return stock_data
    except Exception as e:
        print(f"An error occurred for ticker {ticker}: {e}")
        missing_data_tickers.append(ticker)
        return None


In [5]:
# for a variety of periods load in different list of tickers
def download_stock_data_for_periods(tickers, periods):
    all_data = {}
    
    for period, (start_date, end_date) in periods.items():
        period_data = {}
        for ticker in tickers:
            data = get_data_from_start_to_end(ticker, start_date, end_date)
            if data is not None:
                period_data[ticker] = data
        all_data[period] = period_data
    
    return all_data

In [6]:
import pandas as pd

# Get the adjusted close prices
adj_close_sector_etf = {}

# Create adjusted close price only listing of sector ETFs
def get_adjusted_closed_price(nested_dict, tickers, periods):
    for period in periods:
        stock_price_df = pd.DataFrame()  # Create a new DataFrame for each period
        for ticker in tickers:
            stock_price_df[ticker] = nested_dict[period][ticker]['Adj Close']
        
        adj_close_sector_etf[period] = stock_price_df  # Store the complete DataFrame for the period
    
    return adj_close_sector_etf

##### Plot go graph

In [17]:
# reusable function to create a time series plot

def plot_time_series(data,title,x_label,y_label):
    figure=go.Figure()

    for col in data.columns:
        figure.add_trace(go.Scatter(
            x=data.index,
            y=data[col],
            mode='lines',
            name='col'
        ))

    figure.update_layout(
        title=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
)

    return figure


# Buy and Hold Strategy
The buy and hold investment approach is where an investor purchases stocks or other assets and holds onto them. The period of holding a stock depends on its current performance and projections on the overall movement of the data. This strategy does not look at volatility as a factor as is a popular approach taken by many passive investors as it is build off the assumptions that over time the movement of the stock market is positive. Many investors like the simplicity and reduction in stress of frequest trading.

## Sector ETFs
ETFs are exchange traded funds which are the representation of an accumulation of a variety of stocks. They will try to represent a particular variable such a sector etf. This is where the ETF will take a proportion of a variety sized stocks in one of the 11 GICS Sectors (see documentation) to understand the overall movement of the sector. This looks to mitigate the impact of an individual companies success in understanding a sector. Understading sector performance are important for investing and macroeconomics. For example during a downturn sectors such as healthcare and consumer staples remain strong or outperform most benchmarks as they remain in demand, these are referred to as defensive stocks.

Sector ETFs have only become widely available recently so their history does not go back much further than 2005.

## Macroeconomic Cycle
The business cycle represents the overall aggregate demand within an economy and is very important for investing and macroeconomic policy. It is essential for investing as when the economy is in an expansion (increase) then businesses are increasing profits which increase share value. However, if you think back to Covid-19 or the Global Financial Crises when the economy was in freefall the majority of stocks did not perform well due to decreased demand due to large unemployment rates. All of these systems are related so understanding the performance of the economy is essential for looking at investment strategies. 

### Buy and Hold Strategies using Sector ETFs during different Macroecnomic Cycles
- Long term buy and hold: When the buy and hold period stretches greater than 10 years the success of the investment is less dependent on the business cycle. This is because within a 10 year period there will have been a number of movements but the assumption is that there is an increase in market with time.

- Mid term buy and hold: The investment strategy that will be looked at here will be one where the hold period is between 6 months and 3 years. The sector ETFs do not have historical data going back decades so data will be looked at from 2005-2024. Where there was a number of macroeconomic cycles.

In [1]:
# create time periods for where this takes place
economic_cycle_periods = {

    "trough": ("2008-10-01", "2009-06-01"),
    "expansion": ("2012-01-01", "2015-01-01"),
    "peak": ("2019-06-01", "2020-02-01"),
    "contraction": ("2007-12-01", "2008-10-01"),
    'all_data': ('2005-01-01','2024-06-01')
}

economic_cycle_periods_list = ['trough','expansion','peak','contraction','all_data']

In [2]:
# create etf tickers for sectors
sector_etf_tickers = [
    'XLB', # materials sector
    'XLI', # industrials sector
    'XLF', # financials
    'XLK', # information technology
    'XLY', # consumer discretionary
    'XLP', # consumer staples
    'XLE', # energy
    'XLV', # healthcare
    'VOX', # communication services
    'XLU', # utilities
    'IYR' # real estate
    ]

### Load in Sector ETF Data
Using different macroeconomic cycles and different sector ETFs load in the data via nested dictionary. This is going to allow you to access all relevant candlestick data via both the cycle and the sector etf.

In [8]:
# use down_stock_data_for_periods to get the neccessary stock data to be stored in a nested dict
sector_etf_data = download_stock_data_for_periods(sector_etf_tickers,economic_cycle_periods)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [12]:
# practice example - access technology (XLK) sector ETF during an expansion
sector_etf_data['expansion']['XLK']

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,25.870001,26.020000,25.790001,25.809999,21.622313,26839900
2012-01-04,25.799999,25.930000,25.660000,25.879999,21.680956,4744300
2012-01-05,25.850000,26.010000,25.730000,25.959999,21.747971,6756900
2012-01-06,25.980000,26.030001,25.850000,25.969999,21.756355,4989200
2012-01-09,26.059999,26.080000,25.860001,25.930000,21.722841,4086600
...,...,...,...,...,...,...
2014-12-24,42.279999,42.279999,42.130001,42.139999,37.268230,2655600
2014-12-26,42.220001,42.400002,42.200001,42.320000,37.427429,2278500
2014-12-29,42.160000,42.279999,42.070000,42.110001,37.241714,4638300


### Use of Data
As this investment strategy is used for long term stock data price, the only relevant data is going to be the adjusted close. The adjusted close is used instead of close as it incorporates stock splits which can heavily impact the close price. As a result create a single dataframe where it looks at each sector etfs adjusted close price with different periods.

In [13]:
# using the function 'get_adjusted_closed_price' get the adjusted close data
sector_etf_adjusted_close = get_adjusted_closed_price(sector_etf_data,sector_etf_tickers,economic_cycle_periods_list)

In [15]:
# show the adjusted close price for all sector ETFs durign an expansion
sector_etf_adjusted_close['expansion']

,XLB,XLI,XLF,XLK,XLY,XLP,XLE,XLV,VOX,XLU,IYR
Date,,,,,,,,,,,
2012-01-03,26.488382,27.099348,8.536140,21.622313,33.701874,23.113380,45.198643,28.458916,48.500954,22.923895,37.358257
2012-01-04,26.711489,27.225073,8.510544,21.680956,33.967369,23.063534,45.294056,28.321243,47.982105,22.794167,36.939968
2012-01-05,26.788437,27.232922,8.625728,21.747971,34.250015,23.049292,45.077774,28.402227,47.571674,22.839569,37.207928
2012-01-06,26.749956,27.177925,8.574533,21.756355,34.301403,22.899752,44.810589,28.450813,47.501976,22.716318,37.142567
2012-01-09,26.803818,27.382208,8.619328,21.722841,34.250015,22.935360,45.026875,28.499413,47.579414,22.748758,37.038002
...,...,...,...,...,...,...,...,...,...,...,...
2014-12-24,40.311272,47.958057,16.805046,37.268230,64.257965,38.128925,53.998051,58.572617,71.317329,34.871941,56.719673
2014-12-26,40.450970,47.958057,16.805046,37.427429,64.606819,38.190681,53.984589,58.990730,71.784172,35.293571,56.939289
2014-12-29,40.450970,48.033245,16.865660,37.241714,65.098763,38.074867,54.180012,59.195553,71.800819,35.700665,57.188179


In [34]:
# set up dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Label('Economic Time Period'),
    dcc.Dropdown(
        id='first-dropdown',
        options=[{'label':period,'value':period} for period in economic_cycle_periods_list],
        value = 'all_data'
    ),
    dcc.Graph(id='sector-etf-graph') # create a graph placeholder
])

@app.callback(
    Output('sector-etf-graph','figure'),
    [Input('first-dropdown','value')]
)

def update_graph(selected_period):
    # get the data for the selected period
    filtered_data = sector_etf_adjusted_close[selected_period]

    # use the fig creation 'plot_time_series' to create a visualization
    fig = plot_time_series(filtered_data,f'{selected_period} Sector ETF Adjusted Close','Date','Sector ETF price')

    return fig

# display the app
if __name__ == '__main__':
    app.run_server(port=8052)